# Building A Chatbot

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001D4F703ED50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D4F703F750>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage
model.invoke([HumanMessage(content="Hi, My name is Kashyap and I am a Research Scholar")])

AIMessage(content="Hello Kashyap, nice to meet you. Congratulations on being a Research Scholar. That's a significant achievement. What field of research are you currently working in, and what's the focus of your study? I'd love to learn more about your work.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 49, 'total_tokens': 102, 'completion_time': 0.176472558, 'prompt_time': 0.002225954, 'queue_time': 0.03880914, 'total_time': 0.178698512}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e59d9a5b-0533-477b-bce7-fe27321147bf-0', usage_metadata={'input_tokens': 49, 'output_tokens': 53, 'total_tokens': 102})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi My name is Kashyap and I am a Research Scholar"),
    AIMessage(content="Nice to meet you, Kashyap. Congratulations on being a Research Scholar. That's a significant achievement. What field of research are you currently working in? I'd love to know more about your area of interest and what you're working on."),
    HumanMessage(content="Hey whats my name and what do I do?")
])

AIMessage(content="Your name is Kashyap, and you're a Research Scholar.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 118, 'total_tokens': 133, 'completion_time': 0.03073829, 'prompt_time': 0.005039998, 'queue_time': 0.008484976, 'total_time': 0.035778288}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f4aeab04-0cab-40ac-814a-badf93069718-0', usage_metadata={'input_tokens': 118, 'output_tokens': 15, 'total_tokens': 133})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the mode, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Kashyap and I am a Research Scholar")],
    config=config
)

d:\projects\ml_hub\LangChain\langchain\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


In [8]:
response.content

"Hello Kashyap, nice to meet you. As a research scholar, you must be involved in some interesting projects and studies. What field of research are you currently working in, and what topics are you exploring? I'd be happy to chat with you about your work."

In [9]:
with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config
)

AIMessage(content="Your name is Kashyap, and you mentioned that you're a Research Scholar.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 117, 'total_tokens': 135, 'completion_time': 0.057523346, 'prompt_time': 0.005169238, 'queue_time': 0.039512967, 'total_time': 0.062692584}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--4d425044-f7bf-4556-82d1-abb7d474e9d5-0', usage_metadata={'input_tokens': 117, 'output_tokens': 18, 'total_tokens': 135})

In [10]:
## Change the config --> session_id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config1
)
response.content

"I don't know your name. I'm a large language model, I don't have the ability to know personal information about you, including your name, unless you tell me. If you'd like to share your name, I'd be happy to chat with you and address you by name."

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey my name is John")],
    config=config1
)
response.content

"Nice to meet you, John. It's great to have a name to associate with our conversation. How's your day going so far, John? Is there anything on your mind that you'd like to talk about or any questions you have? I'm here to help and chat with you."

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config1
)
response.content

'I remember, your name is John.'

## Prompt templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain=prompt|model


In [14]:
chain.invoke(
    {"messages":[HumanMessage(content="Hi my name is Kashyap")]}
)

AIMessage(content="Hello Kashyap, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 59, 'total_tokens': 87, 'completion_time': 0.0510937, 'prompt_time': 0.001937332, 'queue_time': 0.008751282, 'total_time': 0.053031032}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--d9b5a8d7-30ac-4e17-9d1f-febcb72e3df5-0', usage_metadata={'input_tokens': 59, 'output_tokens': 28, 'total_tokens': 87})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Kashyap")],
    config=config
)
response.content

"Hello Kashyap, it's nice to meet you. Is there something I can help you with or would you like to chat?"

In [17]:
## Add more complex
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain=prompt|model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Kashyap")], "language":"Telugu"})

In [19]:
response.content

'నమస్తే కశ్యప్, నేను మీకు ఎలా సహాయం చేయగలను?'

Let's now wrap this more complicated chain in a message history class. This time because there are multiple keys in the input we need to specify the correct key to use the save the chat history

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I am Kashyap")],"language": "Hindi"},
    config=config
)
response.content

'नमस्ते कश्यप, मैं आपका सहायक हूँ। मैं आपके प्रश्नों का उत्तर देने के लिए तैयार हूँ। क्या आप कुछ पूछना चाहते हैं?'

In [22]:
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name?")],"language": "Telugu"},
    config=config
)
response.content

'మీ పేరు కశ్యప్.'

## Managing the Conversation History
`trim_messages` helper to reduce how many message we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [31]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi! I am Kashyap"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks!"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)
response= chain.invoke(
    {"messages": messages + [HumanMessage(content="What ice cream do I like")],
    "language": "English"}
)
response.content

"I don't have that information. I'm a text-based AI assistant, and we just started chatting, so I don't know your personal preferences, including your favorite ice cream flavor. Would you like to share?"

In [33]:
response= chain.invoke(
    {"messages": messages + [HumanMessage(content="What math problem did i ask")],
    "language": "English"}
)
response.content

'You asked me to solve the math problem "2 + 2".'

In [35]:
# Lets wrap this in the message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "English"
    },
    config=config
)
response.content

"I don't know your name. You haven't told me what it is. Would you like to share it with me?"